In [47]:
import pandas_gbq as pdgbq
import pandas as pd
import plotly.express as px

In [48]:
inningRuns ='''
-- Temp function to get decimals rounded

CREATE TEMP FUNCTION decround (x FLOAT64) AS (
  ROUND(x,2)
);

WITH rawruns AS (SELECT
    Inning,
    CONCAT(inning_topbot,inning) AS Inninghalf,
    SUM((post_bat_score - bat_score)) AS runsScored
FROM 
    `valuesheet.MLB.hitting_data`
GROUP BY 
    Inning,
    Inninghalf
)

SELECT 
    Inning,
    Inninghalf,
    runsScored,
    SUM(runsScored) OVER (PARTITION BY inning) AS inningruns,
    decround((runsScored/(SELECT SUM(runsScored) FROM rawruns))*100) AS pctrunshalf,
    decround((SUM(runsScored) OVER (PARTITION BY inning))/(SELECT SUM(runsScored) FROM rawruns)*100) AS pctrunsinning,
FROM 
    rawruns
WHERE
    Inning <= 9
ORDER BY
    Inning,
    Inninghalf --ordering alpahabetically on inning half so I can get the top of the inning at the top of the bar chart and the bottom of the inning at the top of the bar chart. 
'''

In [49]:
inningRunsTable = pdgbq.read_gbq(query=inningRuns,project_id='valuesheet')

In [ ]:
inningRunsTable

,Inning,Inninghalf,runsScored,inningruns,pctrunshalf,pctrunsinning
0,1,Bot1,9532,17524,6.68,12.28
1,1,Top1,7992,17524,5.60,12.28
2,2,Bot2,7577,14469,5.31,10.14
3,2,Top2,6892,14469,4.83,10.14
4,3,Bot3,8536,16378,5.98,11.48
5,3,Top3,7842,16378,5.50,11.48
6,4,Bot4,8610,16474,6.03,11.55
7,4,Top4,7864,16474,5.51,11.55
8,5,Bot5,8768,16629,6.15,11.65
9,5,Top5,7861,16629,5.51,11.65


In [ ]:
fig = px.bar(inningRunsTable, x="Inning", y="pctrunshalf", text_auto=True,barmode='stack',title="Distribution of Runs <br><sup>Regulation game of baseball.</sup><br><sup>Top inning over bottom inning.</sup>", )
fig.show()